In [1]:
import dreaminstancing_diff_gaussian_rasterization as ddgr
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import pyquaternion
import math

In [2]:
device = torch.device("cuda")

# generate a random point cloud
N = 100
means3D = torch.cat(
    [
        torch.rand(N, 1, dtype=torch.float32, device=device) * 2 - 1,
        torch.rand(N, 1, dtype=torch.float32, device=device) * 2 - 1,
        torch.rand(N, 1, dtype=torch.float32, device=device) * 10  + 5,
    ],
    dim=1,
)
means2D = torch.zeros(N, 2, dtype=torch.float32, device=device)
opacities = torch.ones(N, 1, dtype=torch.float32, device=device)
scales = torch.ones(N, 3, dtype=torch.float32, device=device)
rotations = (
    torch.from_numpy(np.stack([pyquaternion.Quaternion().elements for _ in range(N)]))
    .to(torch.float32)
    .to(device)
)
colors_precomp = torch.rand(N, 3, dtype=torch.float32, device=device)

RuntimeError: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero.

In [ ]:
def gen_perspective_matrix(
    fovx: float, fovy: float, near: float, far: float
):
    """Generate perspective projection matrix.

    Args:
        fovx (float): field of view angle in degrees in the x direction
        fovy (float): field of view angle in degrees in the y direction
        near (float): near plane
        far (float): far plane

    Returns:
        torch.Tensor: perspective projection matrix
    """
    # Convert fov from degrees to radians and calculate the tangent
    tan_half_fovx = math.tan(math.radians(fovx / 2))
    tan_half_fovy = math.tan(math.radians(fovy / 2))

    return torch.tensor(
        [
            [1 / tan_half_fovx, 0, 0, 0],
            [0, 1 / tan_half_fovy, 0, 0],
            [0, 0, -(far + near) / (far - near), -2*(far * near) / (far - near)],
            [0, 0, -1, 0],
        ]
    )

In [ ]:
FoVx = 90
FoVy = 90

rasterizer = ddgr.GaussianRasterizer(
    ddgr.GaussianRasterizationSettings(
        image_height=512,
        image_width=512,
        tanfovx=math.tan(FoVx * 0.5),
        tanfovy=math.tan(FoVy * 0.5),
        bg=torch.ones(4, dtype=torch.float32, device=device)*5,
        scale_modifier=1.0,
        viewmatrix=torch.eye(4, dtype=torch.float32, device=device),
        projmatrix=gen_perspective_matrix(FoVx, FoVy, 0.1, 100).to(device),
        sh_degree=1,
        campos=torch.tensor([0, 0, 0], dtype=torch.float32, device=device),
        prefiltered=False,
        debug=True,
    )
)

rendered_image, radii, depth, alpha = rasterizer(
    means3D=means3D,
    means2D=means2D,
    colors_precomp=colors_precomp,
    opacities=opacities,
    scales=scales,
    rotations=rotations,
)


# display the rendered image
plt.imshow(rendered_image.permute(1, 2, 0).cpu().numpy())
plt.show()
depth = (depth - depth.min()) / (depth.max() - depth.min())
plt.imshow(depth.permute(1, 2, 0).cpu().numpy(), cmap="gray")
plt.show()

RuntimeError: CUDA error: unspecified launch failure
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
